<a href="https://colab.research.google.com/github/kevin-eschbach/notebooks/blob/main/image_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
import string
from google.colab.patches import cv2_imshow
import random
from copy import deepcopy
import shutil
import numpy as np
from skimage.util import random_noise

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
img_path = '/content/gdrive/MyDrive/Uni/22WT/AutonomousVehicles/img/' # change according to your folder structure

In [ ]:
# Augmentation modes
NOISE_SP = 0
NOISE_GAUSS = 1
BLURRING = 2
BRIGHTNESS_CHANGE = 3

In [ ]:
def noise_sp(image):
  amount = random.random() * .5
  noise_img = random_noise(image, mode='pepper',amount=amount)
  noise_img = np.array(255*noise_img, dtype = 'uint8')
  return noise_img

def noise_gauss(image):
  mean = 0
  var = random.random() * .2
  noise_img = random_noise(image, mode='gaussian', mean=mean, var=var)
  noise_img = np.array(255*noise_img, dtype = 'uint8')
  return noise_img


def blurring(image):
  return cv2.GaussianBlur	(src=image, dst=image,
                           ksize = (11,11),
                           sigmaX=random.random() * 10,
                           sigmaY=random.random() * 10,
                           borderType = cv2.BORDER_REFLECT)
  
def brightness_change(image):
  value = random.randint(-60, 60)
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  h, s, v = cv2.split(hsv)

  if value > 0:
    upper = 255 - value
    v[v > upper] = 255
    v[v <= upper] += value
  else:
    lower = abs(value)
    v[v < lower] = 0
    v[v >= lower] -= abs(value)

  final_hsv = cv2.merge((h, s, v))
  return cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

In [ ]:
counter = 0
chance = .75
for file in os.listdir(img_path):
  if not file.endswith('.txt'):
    file_name = file.split('.')
    file_name = '.'.join(file_name[:len(file_name) - 1])
    file_type = file.split('.')[-1] if len(file.split('.')) > 1 else 'dir'
    if file_type in ['png', 'jpeg', 'jpg']:
      original = cv2.imread(img_path + file, cv2.IMREAD_UNCHANGED)
      augmented = deepcopy(original)
      if original is None:
        print('error')
      else:
        aug_mode = random.choice(range(4))
        if aug_mode == NOISE_SP:
          augmented = noise_sp(augmented)
        elif aug_mode == NOISE_GAUSS:
          augmented = noise_gauss(augmented)
        elif aug_mode == BLURRING:
          augmented = blurring(augmented)
        elif aug_mode == BRIGHTNESS_CHANGE:
          augmented = brightness_change(augmented)
        
        if (chance - random.random()) > 0:
          # cutout part of image
          pt1 = (random.randint(0, augmented.shape[0]),
                 random.randint(0, augmented.shape[1]))
          sizeX = random.randint(20, 100)
          sizeY = random.randint(20, 100)
          pt2 = (pt1[0] + sizeX, pt1[1] + sizeY)
          if random.random() > .5:
            cv2.rectangle(augmented, pt1, pt2, color=(0,0,0), thickness=-1)
          else:
            cv2.circle(augmented,pt1, sizeX, color=(0,0,0), thickness=-1 )


        cv2.imwrite('/content/gdrive/MyDrive/Uni/22WT/AutonomousVehicles/augmented/' +'aug_' + file, augmented)
        shutil.copy2(img_path + file_name + '.txt', 
                     '/content/gdrive/MyDrive/Uni/22WT/AutonomousVehicles/augmented/'+ 'aug_' + file_name + '.txt')
        
        counter += 1
        if (counter % 100) == 0:
          print(f'augmented {counter} files')
    


In [ ]:
# To filter out unlabeled images
# for file in os.listdir(img_path):
#   if not file.endswith('.txt'):
#     file_name = file.split('.')
#     file_name = '.'.join(file_name[:len(file_name) - 1])
#     file_type = file.split('.')[-1] if len(file.split('.')) > 1 else 'dir'
#     if file_type in ['png', 'jpeg', 'jpg']:
#       try:
#         shutil.copy2(img_path + file_name + '.txt', 
#                      '/content/gdrive/MyDrive/Uni/22WT/AutonomousVehicles/test/'+ 'aug_' + file_name + '.txt')
#       except Exception as e:
#         print(str(e))